# Quantization Neural Network with Eve

In this script, we will build a quantization neural network with eve-mli,
different kinds of quantization function will be compared.

In [1]:
# import necessary packages.
# at the beginning, ensure that the eve-mli package is in your python path.
# or you just install it via `pip install eve-mli`.

import os
import time
from datetime import datetime

import random
import numpy as np
import torch as th
import torch.nn as nn
import torch.nn.functional as F

import eve
import eve.app
import eve.app.model
import eve.app.trainer
import eve.core
import eve.app.space as space
import eve.core.layer
import eve.core.quan

from matplotlib import pyplot as plt
%matplotlib inline

os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
# build a basic network for trainer, use Poisson Encoder as default
class mnist(eve.core.Eve):
    def __init__(self,
                 bits: int = 8,
                 quantize_fn: str = "Round",
                 range_tracker: str = "average_tracker",
                 average_tracker_momentum: float = 0.1,
                 upgrade_bits: bool = False,
                 neuron_wise: bool = False,
                 asymmetric: bool = False,
                 signed_quantization: bool = False,
                 learnable_alpha: bool = None,
                 state_list: list = None,
                 upgrade_fn: callable = None,
                 **kwargs,):
        super().__init__()
 
        def build_act_quantizier(state):
            return eve.core.quan.Quantizer(state, 
                         bits = bits,
                         quantize_fn = quantize_fn,
                         range_tracker = range_tracker,
                         average_tracker_momentum = average_tracker_momentum,
                         upgrade_bits = upgrade_bits,
                         neuron_wise = neuron_wise,
                         asymmetric = asymmetric,
                         signed_quantization = signed_quantization,
                         learnable_alpha = learnable_alpha,
                         state_list = state_list,
                         upgrade_fn = upgrade_fn,
                         **kwargs,)
        
        def build_conv_bn_quantizer(in_channels, out_channels, kernel_size, stride, padding):
            return eve.core.layer.QuanBNFuseConv2d(
                            in_channels=in_channels,
                            out_channels=out_channels,
                            kernel_size=kernel_size,
                            stride=stride, 
                            padding=padding,
                            bits = bits,
                            quantize_fn = quantize_fn,
                            range_tracker = range_tracker,
                            average_tracker_momentum = average_tracker_momentum,
                            upgrade_bits = upgrade_bits,
                            neuron_wise = neuron_wise,
                            asymmetric = asymmetric,
                            signed_quantization = signed_quantization,
                            learnable_alpha = learnable_alpha,
                            state_list = state_list,
                            upgrade_fn = upgrade_fn,
                            **kwargs,)

        self.conv1 = build_conv_bn_quantizer(1, 4, 3, stride=2, padding=1)
        self.act_quan1 = build_act_quantizier(eve.core.State(self.conv1))

        self.conv2 = build_conv_bn_quantizer(4, 8, 3, stride=2, padding=1)
        self.act_quan2 = build_act_quantizier(eve.core.State(self.conv2))

        self.conv3 = build_conv_bn_quantizer(8, 16, 3, stride=2, padding=1)
        self.act_quan3 = build_act_quantizier(eve.core.State(self.conv3))

        self.linear1 = eve.core.layer.QuanLinear(16 * 4 * 4, 16, **kwargs)
        self.act_quan4 = build_act_quantizier(eve.core.State(self.linear1))
        
        self.linear2 = nn.Linear(16, 10)

    def forward(self, x):
        conv1 = self.conv1(x)
        act_quan1 = self.act_quan1(conv1)

        conv2 = self.conv2(act_quan1)
        act_quan2 = self.act_quan2(conv2)

        conv3 = self.conv3(act_quan2)
        act_quan3 = self.act_quan3(conv3)

        act_quan3 = th.flatten(act_quan3, start_dim=1).unsqueeze(dim=1)

        linear1 = self.linear1(act_quan3)
        act_quan4 = self.node4(linear1)

        linear2 = self.linear2(act_quan4)

        return linear2.squeeze(dim=1)

In [3]:
# define a MnistClassifier
# Classifier uses the corss entropy as default.
# in most case, we just rewrite the `prepare_data`.
class MnistClassifier(eve.app.model.Classifier):
    def prepare_data(self, data_root: str):
        from torch.utils.data import DataLoader, random_split
        from torchvision import transforms
        from torchvision.datasets import MNIST

        train_dataset = MNIST(root=data_root,
                              train=True,
                              download=True,
                              transform=transforms.ToTensor())
        test_dataset = MNIST(root=data_root,
                             train=False,
                             download=True,
                             transform=transforms.ToTensor())
        self.train_dataset, self.valid_dataset = random_split(
            train_dataset, [55000, 5000])
        self.test_dataset = test_dataset

        self.train_dataloader = DataLoader(self.train_dataset,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=4)
        self.test_dataloader = DataLoader(self.test_dataset,
                                          batch_size=128,
                                          shuffle=False,
                                          num_workers=4)
        self.valid_dataloader = DataLoader(self.valid_dataset,
                                           batch_size=128,
                                           shuffle=False,
                                           num_workers=4)

In [4]:
# store accuracy result
y = {}
def plot():
    global y
    keys, values = list(y.keys()), list(y.values())
    for k, v in y.items():
        plt.plot(v, 
                 color='green' if random.random() > 0.5 else "red", 
                 marker='o' if random.random() > 0.5 else "*", 
                 linestyle='-' if random.random() > 0.5 else ":", 
                 label=k)
    plt.title('accuracy over epoches (train)')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.legend(loc="upper left")
    plt.show()

In [5]:
def train(net, exp_name: str = "quan", data_root: str = "home/densechen/dataset"):
    global y
    # replace the data_root for your path.
    classifier = MnistClassifier(net)
    classifier.prepare_data(data_root=data_root)
    
    # use default configuration
    classifier.setup_train()
    
    # assign model to trainer
    eve.app.trainer.BaseTrainer.assign_model(classifier)
    
    trainer = eve.app.trainer.BaseTrainer()
    
    # train 10 epoches and report the final accuracy
    y[exp_name] = []
    tic = datetime.now()
    for _ in range(10):
        info = trainer.fit()
        y[exp_name].append(info["acc"])
    info = trainer.test()
    toc = datetime.now()
    y[exp_name] = np.array(y[exp_name])
    print(f"Test Accuracy: {info['acc']*100:.2f}%, Elapsed time: {toc-tic}")

## Round vs LSQ vs LLSQ

In [6]:
# reset y
y = {}

# define quantization neural network with differnt quantization function
quantization_neural_network_round = mnist(quantize_fn="Round").quantize()
quantization_neural_network_lsq = mnist(quantize_fn="lsq").quantize()
quantization_neural_network_llsq = mnist(quantize_fn="llsq").quantize()

print("===> Quantization with Round")
train(quantization_neural_network_round, "Round")

print("===> Quantization with lsq")
train(quantization_neural_network_lsq, "lsq")

print("===> Quantization with llsq")
train(quantization_neural_network_llsq, "llsq")

plot()

===> Quantization with Round


RuntimeError: Given groups=1, weight of size [4, 4, 3, 3], expected input[128, 1, 28, 28] to have 4 channels, but got 1 channels instead